This notebook runs cross-validation for all clustering methods so we can compare them in one handy location. 

### imports

In [ ]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np

import emission.storage.timeseries.abstract_timeseries as esta
import emission.storage.decorations.trip_queries as esdtq
import emission.core.get_database as edb
from performance_eval import get_clf_metrics, cv_for_all_algs

### load data

In [ ]:
all_users = esta.TimeSeries.get_uuid_list()
confirmed_trip_df_map = {}
labeled_trip_df_map = {}
expanded_labeled_trip_df_map = {}
expanded_all_trip_df_map = {}
for u in all_users:
    ts = esta.TimeSeries.get_time_series(u)
    ct_df = ts.get_data_df("analysis/confirmed_trip")

    confirmed_trip_df_map[u] = ct_df
    labeled_trip_df_map[u] = esdtq.filter_labeled_trips(ct_df)
    expanded_labeled_trip_df_map[u] = esdtq.expand_userinputs(
        labeled_trip_df_map[u])
    expanded_all_trip_df_map[u] = esdtq.expand_userinputs(
        confirmed_trip_df_map[u])

### run evaluations

In [ ]:
cv_results = cv_for_all_algs(
    uuid_list=all_users,
    expanded_trip_df_map=expanded_labeled_trip_df_map,
    model_names=[
        'old clustering', 'new clustering by combo r100m',
        'new clustering by end r100m', 'new clustering by trip r100m',
        'new clustering by combo r150m', 'new clustering by end r150m',
        'new clustering by trip r150m',
        'random forest with end and trip r150m',
        'random forest with end and trip r100m',
        'random forest with end and trip, drop unclustered r150m',
        'random forest with end r100m',
        'random forest with end and trip, drop unclustered r100m',
        'random forest with start and end r100m',
        'random forest with start end trip r100m', 'adaboost basic', 'random forest, no clustering'
    ],
    override_prior_runs=False,
    raise_errors=False,
    random_state=42,
)


In [ ]:
# store results for all models in a nice dataframe
all_model_results = {
    ('model_name', ''): [],
    ('trips without prediction', 'mode'): [],
    ('trips without prediction', 'purpose'): [],
    ('trips without prediction', 'replaced'): [],
    ('accuracy overall', 'mode'): [],
    ('accuracy overall', 'purpose'): [],
    ('accuracy overall', 'replaced'): [],
    ('accuracy of trips w predictions', 'mode'): [],
    ('accuracy of trips w predictions', 'purpose'): [],
    ('accuracy of trips w predictions', 'replaced'): [],
    ('f1 weighted', 'mode'): [],
    ('f1 weighted', 'purpose'): [],
    ('f1 weighted', 'replaced'): [],
}

for model_name in cv_results.keys():
    print(f'now evaluating: {model_name}')
    all_model_results[('model_name', '')] += [model_name]
    for label_type in ['mode', 'purpose', 'replaced']:
        # get results
        results = get_clf_metrics(cv_results[model_name],
                                  label_type,
                                  keep_nopred=True,
                                  ignore_custom=False)

        # update our dict of aggregate results
        all_model_results[('trips without prediction', label_type)] += [
            results['n_trips_without_prediction']
        ]
        all_model_results[('accuracy overall',
                           label_type)] += [results['accuracy']]
        all_model_results[('accuracy of trips w predictions', label_type)] += [
            results['accuracy'] * len(results['label_true']) /
            (len(results['label_true']) -
             results['n_trips_without_prediction'])
        ]
        all_model_results[('f1 weighted',
                           label_type)] += [results['weighted_f_score']]

all_model_results_df = pd.DataFrame(all_model_results)
all_model_results_df
